In [3]:
from IPython.display import display, HTML


Import DS modules

In [53]:
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta
import statsmodels.api as sm

## Getting Started

In this notebook, we will walk through what it takes to build and assess quality of a simple strategy

We will rely heavily on `TDSCoinbaseData`, `TDSTickGenerator`, and `TDSTransactionTracker`

In [4]:
from TDSCoinbaseData import TDSCoinbaseData
from TDSTickGenerator import TDSTickGenerator
from TDSTransactionTracker import TDSTransactionTracker
import logging
logging.getLogger().setLevel(level=logging.ERROR)

In [221]:
# instantiate our data interface
cb_obj = TDSCoinbaseData(cache_path='data', notebook_logging=True)

In [222]:
# Set the start and end date
start_date = '20201001'
end_date = '20201231'

# List all of the products to be used in the strategy. Be sure to list all products you may use, these can't be updated later
products = ['BTC-USD', 'ETH-USD', 'BTC-EUR']

In [223]:
# Instantiate our transaction tracker. As required, we start with 1.0 BTC
trans_tracker = TDSTransactionTracker(start_date, end_date, holdings={'BTC' : 1.0})

In [224]:
# Instantiate a tick generator
tick_gen = TDSTickGenerator(cb_obj, products, start_date, end_date, interval=60)

## Init Pandas Dataframe aka timeseries in this setting.

In [225]:
d = {'open': [], 'close': [], 'volume': [], 'datetime': []}
# history dataframes
btc_usd = pd.DataFrame(data=d)
eth_usd = pd.DataFrame(data=d)
#ltc_usd = pd.DataFrame(data=d)
#bch_usd = pd.DataFrame(data=d)
# prediction
pred_btc = None
pred_eth = None
# TODO: The flag for normal/low/high volatility period

starttime = '2020-01-01 00:00:00+00:00'
holding_btc = True;
size = 0
MAX_TRAINNING_SIZE = 1440
SIZE_LIMIT_REACHED = False

## Loads Initial data (for testing only)
## The model (probably) doesn't work on small sample size


In [230]:
# Appends value for a new tick to existing dataframes
# Input: 
#    tick -- the tick obj
#    btc_usd -- dataframe
#    eth_usd -- dataframe
#    scale_btc -- the scale value for btc/dollar
#    eth_btc -- the scale value for eth/dollar
def append_ticks(tick, btc_usd, eth_usd, scale_btc=1000, scale_eth=1000, max_r=SIZE_LIMIT_REACHED):
    t_usd = tick.p.btc_usd
    t_eth = tick.p.eth_usd
    btc_usd = btc_usd.append({'open': scale_btc/t_usd.open, 'close': scale_btc/t_usd.close, 
                      'volume': t_usd.volume, 'datetime': tick.datetime}, ignore_index=True)
    eth_usd = eth_usd.append({'open': scale_eth/t_eth.open, 'close': scale_eth/t_eth.close, 
                      'volume': t_eth.volume, 'datetime': tick.datetime}, ignore_index=True)
    # Checks if size limit has been reached
    if (SIZE_LIMIT_REACHED or len(btc_usd.index) > MAX_TRAINNING_SIZE) :
        btc_usd.iloc[1:]
        max_r = True
    if (SIZE_LIMIT_REACHED or len(eth_usd.index) > MAX_TRAINNING_SIZE) :
        eth_usd.iloc[1:]
        max_r = True
    return(btc_usd, eth_usd, SIZE_LIMIT_REACHED)

#loads initial value for testing
for i in range(0, 150) :
    tick = tick_gen.get_tick()
    (btc_usd, eth_usd, SIZE_LIMIT_REACHED) = append_ticks(tick, btc_usd, eth_usd)


In [231]:
btc_usd

,open,close,volume,datetime
0,0.092635,0.092593,2.646372,2020-10-01 00:03:00+00:00
1,0.092597,0.092443,47.834038,2020-10-01 00:04:00+00:00
2,0.092463,0.092386,1.916095,2020-10-01 00:05:00+00:00
3,0.092397,0.092440,14.504379,2020-10-01 00:06:00+00:00
4,0.092433,0.092459,18.712856,2020-10-01 00:07:00+00:00
...,...,...,...,...
295,0.092537,0.092537,0.968268,2020-10-01 04:58:00+00:00
296,0.092537,0.092541,1.849885,2020-10-01 04:59:00+00:00
297,0.092541,0.092545,0.892457,2020-10-01 05:00:00+00:00
298,0.092545,0.092593,0.841485,2020-10-01 05:01:00+00:00


In [242]:
def calculate_time(pddatetime, numtime, plus=True):
    if plus:
        return pddatetime.to_pydatetime() + timedelta(minutes=numtime)
    else: 
        return pddatetime.to_pydatetime() - timedelta(minutes=numtime)

# frame -- dataframe
# minute -- the number of minutes to be predicted
def predict(frame, minute=3) :
    frame = btc_usd.set_index('datetime')
    y = frame['open'].resample('1T').mean()
    l = frame['close'].resample('1T').mean()
    v = frame['volume'].resample('1T').mean()
    merged = pd.merge(y, l, on='datetime', how="outer")
    frame = pd.merge(merged, v, on='datetime', how="outer")

    mod = sm.tsa.VARMAX(frame[['open', 'close', 'volume']], order=(1,0), error_cov_type='diagonal')
    results = mod.fit(maxiter=50, disp=False)
    print(results.summary().tables[1])
    newtime = calculate_time(tick.datetime, minute)
    pred = results.get_prediction(start=pd.to_datetime(frame.index[0]), end=newtime, dynamic=False)
    pred_ci = pred.conf_int()
    print(pred.predicted_mean)
    return pred

# Call this function for btc prediction
#     minute -- the number of minutes to be predicted
def predict_btc(minute) :
    return predict(btc_usd, minute)
    
def predict_eth(minute) :
    return predict(eth_usd, minute)


## This part below will be the actual while loop
(while is commented out at this point)

In [243]:
import random

tick = tick_gen.get_tick()
(btc_usd, eth_usd, SIZE_LIMIT_REACHED) = append_ticks(tick, btc_usd, eth_usd, SIZE_LIMIT_REACHED)

## For the rest of the trading period we will randomly flip our BTC and USD holdings
# while tick is not None:
    
    # 10% chance of attempting to trade
if random.uniform(0,1) <= 0.1:
    if holding_btc:
        # Check the max amount of BTC we can trade my multiplying the volume by the max_taken_volume (0.5 in our case)
        available_btc = tick.p.btc_usd.volume * trans_tracker.get_max_taken_volume()
        my_btc = trans_tracker.get_holdings()['BTC']
        # Only trade if valid
        if available_btc >= my_btc:
            trans_tracker.make_trade(tick, 'BTC-USD', 'sell', 0.1)
            holding_btc = False
    else:
        # Check the available USD by doing the same operation as above and multiplying by the close price since that is the price that we will execute at
        available_usd = tick.p.btc_usd.volume * trans_tracker.get_max_taken_volume() * tick.p.btc_usd.close
        my_usd = trans_tracker.get_holdings()['USD']
        # Only trade if valid
        if available_usd >= my_usd:
            trans_tracker.make_trade(tick, 'BTC-USD', 'buy', 0.1)
            holding_btc = True

# the int passed in here is the number of minutes about to predict
pred_usd = predict_btc(3)
print(pred_usd.predicted_mean)
print(pred_usd.conf_int())
pred_eth = predict_eth(3)
print(pred_eth.predicted_mean)
print(pred_eth.conf_int())

/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency T will be used.
  % freq, ValueWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


2020-10-01 00:03:00+00:00
Ljung-Box (Q):                 nan, nan, nan   Jarque-Bera (JB):   877495.11, 497362.33, 4505.44
Prob(Q):                       nan, nan, nan   Prob(JB):                        0.00, 0.00, 0.00
Heteroskedasticity (H): 385.92, 538.73, 0.90   Skew:                        -15.53, -13.33, 3.64
Prob(H) (two-sided):        0.00, 0.00, 0.60   Kurtosis:                   264.36, 199.35, 20.40
                               open     close     volume
2020-10-01 00:03:00+00:00  0.093204  0.093096  11.095174
2020-10-01 00:04:00+00:00  0.092241  0.092375   9.380768
2020-10-01 00:05:00+00:00  0.094226  0.093861  18.408906
2020-10-01 00:06:00+00:00  0.091977  0.092178   9.224137
2020-10-01 00:07:00+00:00  0.092633  0.092669  11.744884
...                             ...       ...        ...
2020-10-01 05:06:00+00:00 -0.010804  0.015236   4.549159
2020-10-01 05:07:00+00:00 -0.010850  0.015201   4.355737
2020-10-01 05:08:00+00:00 -0.010844  0.015206   4.382241
2020-10-01 05:09

/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency T will be used.
  % freq, ValueWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/statespace/varmax.py:374: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


2020-10-01 00:03:00+00:00
Ljung-Box (Q):                 nan, nan, nan   Jarque-Bera (JB):   877495.11, 497362.33, 4505.44
Prob(Q):                       nan, nan, nan   Prob(JB):                        0.00, 0.00, 0.00
Heteroskedasticity (H): 385.92, 538.73, 0.90   Skew:                        -15.53, -13.33, 3.64
Prob(H) (two-sided):        0.00, 0.00, 0.60   Kurtosis:                   264.36, 199.35, 20.40
                               open     close     volume
2020-10-01 00:03:00+00:00  0.093204  0.093096  11.095174
2020-10-01 00:04:00+00:00  0.092241  0.092375   9.380768
2020-10-01 00:05:00+00:00  0.094226  0.093861  18.408906
2020-10-01 00:06:00+00:00  0.091977  0.092178   9.224137
2020-10-01 00:07:00+00:00  0.092633  0.092669  11.744884
...                             ...       ...        ...
2020-10-01 05:06:00+00:00 -0.010804  0.015236   4.549159
2020-10-01 05:07:00+00:00 -0.010850  0.015201   4.355737
2020-10-01 05:08:00+00:00 -0.010844  0.015206   4.382241
2020-10-01 05:09

/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/jiaqilu/opt/anaconda3/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)


Debug Mode

In [ ]:
rec = rec.set_index('datetime')
y = rec['open'].resample('1T').mean()
l = rec['close'].resample('1T').mean()
v = rec['volume'].resample('1T').mean()
merged = pd.merge(y, l, on='datetime', how="outer")
rec = pd.merge(merged, v, on='datetime', how="outer")

mod = sm.tsa.VARMAX(rec[['open', 'close', 'volume']], order=(1,0), error_cov_type='diagonal')
results = mod.fit(maxiter=50, disp=False)
print(results.summary().tables[1])
newtime = tick.datetime.to_pydatetime() + timedelta(minutes=3)
pred = results.get_prediction(start=pd.to_datetime('2020-10-01 00:22:00+00:00'), end=newtime, dynamic=False)
pred_ci = pred.conf_int()
print(pred.predicted_mean)

## Visualizing Our Returns

Once we have completed trading, we can visualize our returns by calling `plot_btc_holdings()`

In [19]:
trans_tracker.plot_btc_holdings()

KeyboardInterrupt: 

## Looking at Daily Stats

We can call `get_pct_change_per_day()` to get a dataframe of our daily `BTC` holdings and day to day percent change

`BTC` holding for a single day is derived by taking the shortest path through each product to `BTC` using the last transaction price of a given day

EX:
To get the `BTC` value of an `ETH` holding, we traverse `ETH` -> `USD` -> `BTC`

This logic is all handled for you as a part of `TDSTransactionTracker`

In [20]:
display(trans_tracker.get_pct_change_per_day())

,date,BTC,pct_diff
0,20201001,0.829108,-0.170892
1,20201002,0.702294,-0.152952
2,20201003,0.630000,-0.102939
3,20201004,0.537633,-0.146614
4,20201005,0.454940,-0.153811
...,...,...,...
87,20201227,0.093548,0.066443
88,20201228,0.095426,0.020067
89,20201229,0.094541,-0.009270
90,20201230,0.091634,-0.030746


## Compute Sharpe Ratio

To get out Sharpe ration, we simply call `get_sharpe_ratio()`

This Sharpe ratio is based on the EOD BTC holdings as shown above and uses the BlockFi 1 BTC interest rate as the risk free return

https://blockfi.com/crypto-interest-account

In [21]:
print(trans_tracker.get_sharpe_ratio())

RISK FREE DAILY RETURN : 
0.00016438356164383562
ACTUAL DAILY RETURN : 
-0.024572212328816654
EXCESS STD : 
0.049256698748459345
-0.5021975998997336


## Saving Your Results

To save your trades to a json file in the required format, when you are done trading, simply call `dump_trades("<filepath>")`

In [ ]:
trans_tracker.dump_trades('example_trades.json')